In [28]:
#Import and installing packages to run code 
import geopandas as gpd 
from shapely.geometry import LineString, Point, box
from tqdm.auto import tqdm
import math
import pandas as pd

In [34]:
#USE THIS ONE WHEN USING A COASTSAT FILE
#Navigate to the transect file in the Data folder and adding an azimuth to the file 
lines = gpd.read_file('Data/coastsat_projected.shp')
def get_azimuth(line):
    p1, p2 = line.coords
    azimuth = math.degrees(math.atan2(p2[0]-p1[0], p2[1]-p1[1]))
    if azimuth < 0:
        azimuth += 360
    return azimuth
lines["Azimuth"] = lines.geometry.apply(get_azimuth)
lines["length"] = lines.length

ValueError: too many values to unpack (expected 2)

In [35]:
#USE THIS ONE WHEN USING A DSAS FILE
#Navigate to the transect file in the Data folder and adding an azimuth to the file 
lines = gpd.read_file('Data/coastsat_projected.shp')
def get_azimuth(line):
    p1, p2 = line.coords[1:]
    azimuth = math.degrees(math.atan2(p2[0]-p1[0], p2[1]-p1[1]))
    if azimuth < 0:
        azimuth += 360
    return azimuth
lines["Azimuth"] = lines.geometry.apply(get_azimuth)
lines["length"] = lines.length

In [36]:
#Navigating to the MHWL file in the Data folder
MHWL = gpd.read_file('Data/nz-MHWL.shp')
MHWL = MHWL[MHWL.intersects(box(*lines.total_bounds))]

In [38]:
#This code checks if the transects intersect with the MHWL 
lines["intersect_point"] = lines.intersection(MHWL.unary_union)
lines

,site_id,orientatio,along_dist,along_di_1,beach_slop,cil,ciu,trend,n_points,n_points_n,id,Shape_Leng,geometry,Azimuth,length,intersect_point
0,nzd0182,257.644640,599.821205,1.000000,0.040,0.0371,0.0441,0.823197,477.0,222.0,nzd0182-0000,0.003564,"LINESTRING Z (1728306.368 5916871.610 0.000, 1...",78.560885,1089.661599,"MULTIPOINT Z (1728200.647 5916850.209 0.000, 1..."
1,nzd0182,257.644640,499.821205,0.833284,0.040,0.0389,0.0445,0.863566,477.0,248.0,nzd0182-0001,0.003564,"LINESTRING Z (1728290.413 5916949.734 0.000, 1...",78.560749,1089.671411,"MULTIPOINT Z (1728214.075 5916934.285 0.000, 1..."
2,nzd0182,257.644640,399.821205,0.666567,0.045,0.0430,0.0506,1.083575,477.0,290.0,nzd0182-0002,0.003564,"LINESTRING Z (1728274.458 5917027.859 0.000, 1...",78.560610,1089.681051,"MULTIPOINT Z (1728196.109 5917011.997 0.000, 1..."
3,nzd0182,255.335635,299.821205,0.499851,0.045,0.0400,0.0498,0.808075,477.0,265.0,nzd0182-0003,0.003552,"LINESTRING Z (1728257.089 5917112.254 0.000, 1...",76.260340,1091.119764,"MULTIPOINT Z (1728174.285 5917092.006 0.000, 1..."
4,nzd0182,252.961842,199.830294,0.333150,0.040,0.0367,0.0447,1.219678,477.0,229.0,nzd0182-0004,0.003537,"LINESTRING Z (1728233.153 5917195.223 0.000, 1...",73.895063,1092.847787,"MULTIPOINT Z (1728152.484 5917171.929 0.000, 1..."
5,nzd0182,250.496004,99.830294,0.166433,0.055,0.0461,0.0617,1.317295,477.0,245.0,nzd0182-0005,0.003520,"LINESTRING Z (1728208.695 5917278.315 0.000, 1...",71.437599,1094.907409,"MULTIPOINT Z (1728107.895 5917244.463 0.000, 1..."
6,nzd0182,248.090652,0.000000,0.000000,0.040,0.0379,0.0463,1.066479,477.0,197.0,nzd0182-0006,0.003502,"LINESTRING Z (1728177.703 5917358.894 0.000, 1...",69.039821,1097.173625,"MULTIPOINT Z (1728109.835 5917332.894 0.000, 1..."
7,nzd0183,244.486130,1199.601493,1.000000,0.030,0.0292,0.0367,2.454733,481.0,239.0,nzd0183-0000,0.003471,"LINESTRING Z (1729033.469 5915341.459 0.000, 1...",65.452123,1100.777439,"MULTIPOINT Z (1728927.918 5915293.246 0.000, 1..."
8,nzd0183,244.486130,1099.601493,0.916639,0.035,0.0334,0.0400,1.502852,481.0,220.0,nzd0183-0001,0.003471,"LINESTRING Z (1729000.084 5915413.908 0.000, 1...",65.451889,1100.786061,"MULTIPOINT Z (1728900.829 5915368.564 0.000, 1..."
9,nzd0183,244.486130,999.601493,0.833278,0.030,0.0285,0.0316,2.000150,481.0,213.0,nzd0183-0002,0.003471,"LINESTRING Z (1728966.699 5915486.358 0.000, 1...",65.451636,1100.794800,"MULTIPOINT Z (1728891.803 5915452.147 0.000, 1..."


In [39]:
#This code defines the funstion that visualises the extended transect segments which will be used later. Ignore the error
def map():
    m = MHWL.explore()
    lines["intersects"] = ~lines.intersect_point.is_empty
    lines.explore("intersects", cmap=["red", "green"], legend=True, m=m)
    lines.intersect_point.explore(m=m, color="magenta")
    lines.intersect_point.dropna()
    display(m)
map()

In [40]:
#Admin code. Run this 
sum(lines.intersect_point.is_empty)

0

In [41]:
#This function calculates the coordinates of the new extended transects. 
#This code makes sure that the MHWL is not intersected twice by the transect, which occurred around the bays and spits so it chooses to extend the transect from the shoreline section it is representing. 
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

def extend(line, distance=-90):
    p1, p2 = line.coords[1:]
    azimuth = get_azimuth(line)
    new_point = calculate_new_coordinates(p1[0], p1[1], azimuth, distance)
    return LineString([new_point, Point(p2[0], p2[1])])

lines.geometry.update(lines[lines.intersect_point.is_empty].geometry.apply(extend))
lines["length"] = lines.length
lines["intersect_point"] = lines.intersection(MHWL.unary_union)
lines

,site_id,orientatio,along_dist,along_di_1,beach_slop,cil,ciu,trend,n_points,n_points_n,id,Shape_Leng,geometry,Azimuth,length,intersect_point,intersects
0,nzd0182,257.644640,599.821205,1.000000,0.040,0.0371,0.0441,0.823197,477.0,222.0,nzd0182-0000,0.003564,"LINESTRING Z (1728306.368 5916871.610 0.000, 1...",78.560885,1089.661599,"MULTIPOINT Z (1728200.647 5916850.209 0.000, 1...",True
1,nzd0182,257.644640,499.821205,0.833284,0.040,0.0389,0.0445,0.863566,477.0,248.0,nzd0182-0001,0.003564,"LINESTRING Z (1728290.413 5916949.734 0.000, 1...",78.560749,1089.671411,"MULTIPOINT Z (1728214.075 5916934.285 0.000, 1...",True
2,nzd0182,257.644640,399.821205,0.666567,0.045,0.0430,0.0506,1.083575,477.0,290.0,nzd0182-0002,0.003564,"LINESTRING Z (1728274.458 5917027.859 0.000, 1...",78.560610,1089.681051,"MULTIPOINT Z (1728196.109 5917011.997 0.000, 1...",True
3,nzd0182,255.335635,299.821205,0.499851,0.045,0.0400,0.0498,0.808075,477.0,265.0,nzd0182-0003,0.003552,"LINESTRING Z (1728257.089 5917112.254 0.000, 1...",76.260340,1091.119764,"MULTIPOINT Z (1728174.285 5917092.006 0.000, 1...",True
4,nzd0182,252.961842,199.830294,0.333150,0.040,0.0367,0.0447,1.219678,477.0,229.0,nzd0182-0004,0.003537,"LINESTRING Z (1728233.153 5917195.223 0.000, 1...",73.895063,1092.847787,"MULTIPOINT Z (1728152.484 5917171.929 0.000, 1...",True
5,nzd0182,250.496004,99.830294,0.166433,0.055,0.0461,0.0617,1.317295,477.0,245.0,nzd0182-0005,0.003520,"LINESTRING Z (1728208.695 5917278.315 0.000, 1...",71.437599,1094.907409,"MULTIPOINT Z (1728107.895 5917244.463 0.000, 1...",True
6,nzd0182,248.090652,0.000000,0.000000,0.040,0.0379,0.0463,1.066479,477.0,197.0,nzd0182-0006,0.003502,"LINESTRING Z (1728177.703 5917358.894 0.000, 1...",69.039821,1097.173625,"MULTIPOINT Z (1728109.835 5917332.894 0.000, 1...",True
7,nzd0183,244.486130,1199.601493,1.000000,0.030,0.0292,0.0367,2.454733,481.0,239.0,nzd0183-0000,0.003471,"LINESTRING Z (1729033.469 5915341.459 0.000, 1...",65.452123,1100.777439,"MULTIPOINT Z (1728927.918 5915293.246 0.000, 1...",True
8,nzd0183,244.486130,1099.601493,0.916639,0.035,0.0334,0.0400,1.502852,481.0,220.0,nzd0183-0001,0.003471,"LINESTRING Z (1729000.084 5915413.908 0.000, 1...",65.451889,1100.786061,"MULTIPOINT Z (1728900.829 5915368.564 0.000, 1...",True
9,nzd0183,244.486130,999.601493,0.833278,0.030,0.0285,0.0316,2.000150,481.0,213.0,nzd0183-0002,0.003471,"LINESTRING Z (1728966.699 5915486.358 0.000, 1...",65.451636,1100.794800,"MULTIPOINT Z (1728891.803 5915452.147 0.000, 1...",True


In [42]:
#Admin code. Run this 
lines.intersect_point.geom_type.value_counts()

MultiPoint    36
Point         18
Name: count, dtype: int64

In [43]:
#This code makes sure that the intersect points are spatially correct and formatted correctly.   
def get_best_intersection(row):
    line_coords = [Point(p) for p in row.geometry.coords]
    p1 = line_coords[-2]
    p2 = line_coords[-1]
    assert p1.distance(p2) > 50
    intersections = gpd.GeoDataFrame({"points": row.intersect_point.geoms}, geometry="points", crs=lines.crs)
    intersections["dist"] = intersections.distance(p1)
    return intersections.points[intersections.dist.idxmin()]

lines.intersect_point.update(lines[lines.intersect_point.geom_type == "MultiPoint"].apply(get_best_intersection, axis=1))

In [44]:
#Visualising the extended transects before exporting them
map()

In [11]:
#Admin code. Run this
assert sum(lines.intersect_point.is_empty) == 0
assert all(lines.intersect_point.geom_type == "Point")

AssertionError: 

In [46]:
#Exporting the extended transects (change the location, name and file type as required) with the MHWL intersect coordinate. 
lines["Intersect_X"] = lines.intersect_point.x
lines["Intersect_Y"] = lines.intersect_point.y
lines.drop(columns="intersect_point").to_file('Data/extended_transects.geojson')